In [1]:
from utils.alloys import alloy_to_1d_tensor, make_params_df, get_elements_and_compositions, element_to_index
from utils.constants import alloy_max_len, n_elements
from utils.ml import *
from utils.dataframes import dfs_tabs
from utils.rnn_predictor import LSTMClassifier, LSTMClassifierBidirectional, save_model_and_see_stats, save_predictions
from utils.ml import run_all_regressors, run_all_regressors_with_transformers
import torch
import matplotlib.pyplot as plt

2023-09-28 22:42:11.733647: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 22:42:12.453695: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-28 22:42:13.877886: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/include:/usr/local/cuda-11.0/lib64:
2023-09-28 22:42:13.878024: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerro

In [2]:
df_train = pd.read_csv("train_final_df.csv")
df_test = pd.read_csv("test_final_df.csv")

print("Training set shape:", df_train.shape)
print("Testing set shape:", df_test.shape)

Training set shape: (1269, 3)
Testing set shape: (318, 3)


In [3]:
# constants
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ideally a max factor is from the training dataset
max_factor = max(df_train["actual_d_max"])
print(max_factor)

35.0


In [ ]:
# creating the input and output data for the model
X_train = []
y_train = []

X_test = []
y_test = []

elements_vector_df = pd.read_csv("utils/elements_wvmodel_2016_12-15-20.csv")

def alloy_to_vectorized_tensor(alloy_str, alloy_max_len = alloy_max_len):
    tensor = torch.zeros(118, 200)
    elements, compositions = get_elements_and_compositions(alloy_str)
    i = 0
    for idx in range(0, len(elements) + len(compositions), 2):
        tensor[element_to_index(elements[i]), :] = torch.tensor(np.array(elements_vector_df[elements[i]])) * compositions[i]
        i += 1
    return tensor

for i in range(df_train.shape[0]):
    X_train.append(alloy_to_vectorized_tensor(df_train.loc[i, "bmg_alloy"]))
    y_train.append(df_train.loc[i, "actual_d_max"] / max_factor)

X_train = torch.stack(X_train)
y_train = torch.tensor(y_train, dtype=torch.float32)

for i in range(df_test.shape[0]):
    X_test.append(alloy_to_vectorized_tensor(df_test.loc[i, "bmg_alloy"]))
    y_test.append(df_test.loc[i, "actual_d_max"] / max_factor)
    
# max_factor we selected is fine
X_test = torch.stack(X_test)
y_test = torch.tensor(y_test, dtype=torch.float32)

print("X_train (shape and min-max):", X_train.shape, min(X_train.view(-1, 1)), max(X_train.view(-1, 1)),
      "y_train (shape and min-max):", y_train.shape, min(y_train.view(-1, 1)), max(y_train.view(-1, 1)))

print("X_test (shape and min-max):", X_test.shape, min(X_test.view(-1, 1)), max(X_test.view(-1, 1)),
      "y_test (shape and min-max):", y_test.shape, min(y_test.view(-1, 1)), max(y_test.view(-1, 1)))